In [1]:
import pyspark
import findspark
findspark.init()
from pyspark.sql import SparkSession

scala_version = '2.12'  # your scala version
spark_version = '3.0.1' # your spark version
packages = [
    f'org.apache.spark:spark-sql-kafka-0-10_{scala_version}:{spark_version}',
    'org.apache.kafka:kafka-clients:2.8.0' #your kafka version
]
spark = SparkSession.builder.master("local").appName("kafka-example").config("spark.jars.packages", ",".join(packages)).getOrCreate()
spark

24/03/15 23:33:05 WARN Utils: Your hostname, ptt-virtual-machine resolves to a loopback address: 127.0.1.1; using 192.168.93.135 instead (on interface ens33)
24/03/15 23:33:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/ptt/anaconda3/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ptt/.ivy2/cache
The jars for the packages stored in: /home/ptt/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.kafka#kafka-clients added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-9b9ee456-7f49-43c8-a597-b24c4039f142;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.0.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.0.1 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.commons#commons-pool2;2.6.2 in central
	found org.apache.kafka#kafka-clients;2.8.0 in central
	found com.github.luben#zstd-jni;1.4.9-1 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.1 in central
	found org.slf4j#slf4j-api;1.7.30 in central
:: resolution report :: resolve 1604ms :: artifacts dl 85ms
	:: modules in use:
	com.github.luben#zstd-jni;1.4.9-1 from central in [default]
	org

In [2]:
from pyspark.sql.functions import *
from pyspark.ml.feature import Normalizer, StandardScaler
import random
import pyspark.sql.functions as f
from pyspark.sql.functions import col
kafka_topic_name = "songTopic10"
kafka_bootstrap_servers = 'localhost:9092'
songs_df = spark.read.format("kafka").option("kafka.bootstrap.servers", kafka_bootstrap_servers).option("subscribe", kafka_topic_name).option("startingOffsets", "earliest").load()
songs_df1 = songs_df.selectExpr("cast(value as string) as value")
songs_df2=songs_df1.select("value", f.translate(f.col("value"), '"', "").alias("values"))
songs_df3 = songs_df2.withColumn('order_id', split(songs_df2['values'], '~').getItem(0).cast("int"))\
                .withColumn("id", split(songs_df2['values'], '~').getItem(1).cast("string"))\
                .withColumn("name", split(songs_df2['values'], '~').getItem(2).cast("string"))\
                .withColumn("popularity", split(songs_df2['values'], '~').getItem(3).cast("int"))\
                .withColumn("duration_ms", split(songs_df2['values'], '~').getItem(4).cast("double"))\
                .withColumn("explicit", split(songs_df2['values'], '~').getItem(5).cast("int"))\
                .withColumn("artists", split(songs_df2['values'], '~').getItem(6).cast("string"))\
                .withColumn("id_artists", split(songs_df2['values'], '~').getItem(7).cast("string"))\
                .withColumn("release_date", split(songs_df2['values'], '~').getItem(8).cast("string"))\
                .withColumn("danceability", split(songs_df2['values'], '~').getItem(9).cast("double"))\
                .withColumn("energy", split(songs_df2['values'], '~').getItem(10).cast("double"))\
                .withColumn("key", split(songs_df2['values'], '~').getItem(11).cast("int"))\
                .withColumn("loudness", split(songs_df2['values'], '~').getItem(12).cast("double"))\
                .withColumn("mode", split(songs_df2['values'], '~').getItem(13).cast("int"))\
                .withColumn("speechiness", split(songs_df2['values'], '~').getItem(14).cast("double"))\
                .withColumn("acousticness", split(songs_df2['values'], '~').getItem(15).cast("double"))\
                .withColumn("instrumentalness", split(songs_df2['values'], '~').getItem(16).cast("double"))\
                .withColumn("liveness", split(songs_df2['values'], '~').getItem(17).cast("double"))\
                .withColumn("valence", split(songs_df2['values'], '~').getItem(18).cast("double"))\
                .withColumn("tempo", split(songs_df2['values'], '~').getItem(19).cast("double"))\
                .withColumn("time_signature", split(songs_df2['values'], '~').getItem(20).cast("double"))


In [3]:
songs_df3.toPandas().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9651 entries, 0 to 9650
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   value             9651 non-null   object 
 1   values            9651 non-null   object 
 2   order_id          9651 non-null   int32  
 3   id                9651 non-null   object 
 4   name              9651 non-null   object 
 5   popularity        9651 non-null   int32  
 6   duration_ms       9651 non-null   float64
 7   explicit          9651 non-null   int32  
 8   artists           9651 non-null   object 
 9   id_artists        9651 non-null   object 
 10  release_date      9651 non-null   object 
 11  danceability      9651 non-null   float64
 12  energy            9651 non-null   float64
 13  key               9651 non-null   int32  
 14  loudness          9651 non-null   float64
 15  mode              9651 non-null   int32  
 16  speechiness       9651 non-null   float64


In [4]:
import os

import numpy as np
import pandas as pd
import ujson
import spotipy
import spotipy.util
import seaborn as sns

os.environ["SPOTIPY_CLIENT_ID"] = 'a997751eb4be4e87bb3a0a246b0a4b1b'
os.environ["SPOTIPY_CLIENT_SECRET"] = '4ee43f4a92fd4bd882fe2667d82ccf50'
os.environ["SPOTIPY_REDIRECT_URI"] = 'http://localhost:5000/callback'

scope = 'user-library-read'
username = 'Tin Pham'

token = spotipy.util.prompt_for_user_token(username, scope)

if token:
    spotipy_obj = spotipy.Spotify(auth=token)
    saved_tracks_resp = spotipy_obj.current_user_saved_tracks(limit=50)
else:
    print('Couldn\'t get token for that username')
    
number_of_tracks = saved_tracks_resp['total']
print('%d tracks' % number_of_tracks)

def save_only_some_fields(track_response):
    return {        
        'id': str(track_response['track']['id']),
        'name': str(track_response['track']['name']),
        'artists': [artist['name'] for artist in track_response['track']['artists']],
        'duration_ms': track_response['track']['duration_ms'],
        'popularity': track_response['track']['popularity'],
        'added_at': track_response['added_at']
    }

tracks = [save_only_some_fields(track) for track in saved_tracks_resp['items']]

while saved_tracks_resp['next']:
    saved_tracks_resp = spotipy_obj.next(saved_tracks_resp)
    tracks.extend([save_only_some_fields(track) for track in saved_tracks_resp['items']])


tracks_df = pd.DataFrame(tracks)
pd.set_option('display.max_rows', len(tracks))


tracks_df['artists'] = tracks_df['artists'].apply(lambda artists: artists[0])
tracks_df['duration_ms'] = tracks_df['duration_ms'].apply(lambda duration: duration/1000)

tracks_df = tracks_df.rename(columns = {'duration_ms':'duration_s'})


audio_features = {}

for idd in tracks_df['id'].tolist():
    audio_features[idd] = spotipy_obj.audio_features(idd)[0]
    
tracks_df['acousticness'] = tracks_df['id'].apply(lambda idd: audio_features[idd]['acousticness'])
tracks_df['speechiness'] = tracks_df['id'].apply(lambda idd: audio_features[idd]['speechiness'])
tracks_df['key'] = tracks_df['id'].apply(lambda idd: str(audio_features[idd]['key']))
tracks_df['liveness'] = tracks_df['id'].apply(lambda idd: audio_features[idd]['liveness'])
tracks_df['instrumentalness'] = tracks_df['id'].apply(lambda idd: audio_features[idd]['instrumentalness'])
tracks_df['energy'] = tracks_df['id'].apply(lambda idd: audio_features[idd]['energy'])
tracks_df['tempo'] = tracks_df['id'].apply(lambda idd: audio_features[idd]['tempo'])
tracks_df['time_signature'] = tracks_df['id'].apply(lambda idd: audio_features[idd]['time_signature'])
tracks_df['loudness'] = tracks_df['id'].apply(lambda idd: audio_features[idd]['loudness'])
tracks_df['danceability'] = tracks_df['id'].apply(lambda idd: audio_features[idd]['danceability'])
tracks_df['valence'] = tracks_df['id'].apply(lambda idd: audio_features[idd]['valence'])
import pandas as pd

song_data = tracks_df
#song_data.rename(columns={'duration_s': 'duration_ms' }, inplace=True)
song_data = song_data.drop(['id', 'added_at', 'time_signature','duration_s'], axis='columns')
add_df = song_data

1 tracks


In [5]:
df = songs_df3


df_stream = df

df = df.drop('value','values','order_id',
 'id',
 'explicit',
  'mode',
 'release_date',
 'id_artists',
 'time_signature',
 'duration_ms',
 'timestamp')

df_sp = spark.createDataFrame(add_df)
df = df.union(df_sp)

from pyspark.ml.feature import VectorAssembler
assembler=VectorAssembler(inputCols=[
 'danceability',
 'energy',
 'loudness',
 'speechiness',
 'acousticness',
 'instrumentalness',
 'liveness',
 'valence',
 'tempo'], outputCol='features')
assembled_data=assembler.setHandleInvalid("skip").transform(df)



In [6]:
from pyspark.ml.feature import StandardScaler
scale=StandardScaler(inputCol='features',outputCol='standardized')
data_scale=scale.fit(assembled_data)
df=data_scale.transform(assembled_data)
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
silhouette_score=[]
evaluator = ClusteringEvaluator(predictionCol='prediction', featuresCol='standardized', \
                                metricName='silhouette', distanceMeasure='squaredEuclidean')


KMeans_algo=KMeans(featuresCol='standardized', k=3)
    
KMeans_fit=KMeans_algo.fit(df)
    
output_df =KMeans_fit.transform(df)

In [9]:
import numpy as np, pandas as pd
import matplotlib.pyplot as plt, seaborn as sns
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

class SpotifyRecommender():
    def __init__(self, rec_data):
        self.rec_data_ = rec_data
    
    def spotify_recommendations(self, song_name, amount=1):
        distances = []
        song = self.rec_data_[(self.rec_data_.name.str.lower() == song_name.lower())].head(1).values[0]
        # get details of our fav song from name we pass as x earlier.
        res_data = self.rec_data_[self.rec_data_.name.str.lower() != song_name.lower()]
        #dropping the data with our fav song so that it doesnt affect our recommendation.
        for r_song in tqdm(res_data.values):
            # tqdm is just used for showing the bar of iteration through our streamed songs.
            dist = 0
            for col in np.arange(len(res_data.columns)):
                # (len(res_data.columns) gets us the number of columns -> 13 in our case.
                #indeces of non-numerical columns neednt be considered.
                if not col in [0,1,13]:
                    #calculating the manhettan distances for each numerical feature
                    # song -> from our fav dataset.
                    # r_song -> from streaming data.
                
                    dist = dist + np.absolute(float(song[col]) - float(r_song[col]))
            distances.append(dist)
            # distances are calculated and appended and added to a new column called distances in our dataset.
        res_data['distance'] = distances
        #sorting our data to be ascending by 'distance' feature
        res_data = res_data.sort_values('distance')
        # resulting dataset have the song similar to our fav song's numerical values and thus recommended.
        columns = ['name', 'artists', 'acousticness', 'liveness', 'instrumentalness', 'energy', 'danceability', 'valence']
        return res_data[columns][:amount]
    

In [29]:
 
datad = output_df.select('name',
 'artists',
 'danceability',
 'energy',
 'key',
 'loudness',
 'speechiness',
 'acousticness',
 'instrumentalness',
 'liveness',
 'valence',
 'tempo',
 'prediction')



datf = datad.toPandas()
datf.drop(datf[datf['artists'] == '0'].index, inplace = True)
datf.drop_duplicates(inplace=True)
datf.drop(datf[datf['danceability'] == 0.0000].index, inplace = True)
datf.drop(datf[datf['liveness'] == 0.000].index, inplace = True)
datf.drop(datf[datf['instrumentalness'] == 0.000000].index, inplace = True)
datf.drop(datf[datf['energy'] == 0.0000].index, inplace = True)
datf.drop(datf[datf['danceability'] == 0.000].index, inplace = True)
datf.drop(datf[datf['valence'] == 0.000].index, inplace = True)

# y = datf
# value_pred = datf.iloc[-1:]['prediction']
#datf = datf[datf['prediction'] == list(value_pred)[0]]
recommender = SpotifyRecommender(datf)
x = add_df['name'].tolist()[0]
print(x)
rec_song = recommender.spotify_recommendations(str(x), 5)

v = add_df[['name', 'artists',  'acousticness', 'liveness', 'instrumentalness', 'energy', 
       'danceability', 'valence']]

rec_song = pd.concat([rec_song, v])

Từ Ngày Em Đến


100%|████████████████████████████████████| 4074/4074 [00:00<00:00, 28716.76it/s]


In [30]:
df_rec = spark.createDataFrame(rec_song)
df_rec.show()

+--------------------+--------------------+------------+--------+----------------+------+------------+-------+
|                name|             artists|acousticness|liveness|instrumentalness|energy|danceability|valence|
+--------------------+--------------------+------------+--------+----------------+------+------------+-------+
|      Sativa - Remix|   ['DJ Alan Gomez']|       0.152|   0.175|         2.46E-4| 0.621|       0.739|  0.618|
|             Oh Love|  ['Emilie Nicolas']|       0.892|  0.0866|         1.31E-6| 0.105|       0.415|  0.323|
|Na Ponta do P\u00...|      ['Mc Livinho']|       0.472|  0.0906|         1.05E-6|  0.76|       0.592|  0.634|
|             Element|       ['Pop Smoke']|      0.0301|   0.251|         2.18E-6| 0.878|       0.772|  0.305|
|Hoy Se Chicha - R...|['DJ Alan Gomez',...|      0.0201|   0.155|         4.95E-5| 0.638|       0.516|   0.61|
|      Từ Ngày Em Đến|              Da LAB|       0.418|   0.105|             0.0|  0.69|       0.699|  0.637|
+